---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [64]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [99]:
def date_sorter():
    
    # Your code here
    
    def func_agg_various_ymd_str(df):
#         print(df['all'])
        # 英語表記Month (これが一番日時として判別しやすい)
        if not pd.isnull(df['word_ymd']):
            mConverter = {'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 'May': '5', 'Jun': '6', 'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
            
            if not pd.isnull(df['wdpre']):                      # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009のタイプ
                return  mConverter[str(df['wm'])[0:3]] + '/'+ df['wdpre'] + '/' +df['wy']
            
            elif not pd.isnull(df['wdpost']):                 # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009, Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;のタイプ
                return mConverter[str(df['wm'])[0:3]]  + '/' + str(int(df['wdpost'].replace('.', ''))) + '/' + df['wy'] # "."は整数変換できないので除去する
            
            else:                                                              # Feb 2009; Sep 2009; Oct 2010のタイプ
                return mConverter[str(df['wm'])[0:3]] + '/1/' + df['wy']                
        # ベーシックスタイル１
        elif not pd.isnull(df['basic_ymd_1']):
            if pd.isnull(df['bfdy']):
                return df['bm_1'].replace('-', '/') + df['bd'].replace('-', '/') + '19' + df['btdy'] # btdy: basic two digit yearの略
            else:
                return df['basic_ymd_1'].replace('-', '/') # dd-mm-yyyyをdd/mm/yyyyに変換する
        # ベーシックスタイル2
        elif not pd.isnull(df['basic_ymd_2']):
            return df['bm_2'].replace('-', '/') + '1/' + df['by']                
        # 年のみ
        elif not pd.isnull(df['year_only']) and int(df['year_only']) > 1900:
                return '1/1/'  + df['year_only']
        # 例外
        else:
            print('...Something wrong!...', df)
            return ''

    # 【1】様々な表記を抽出する。
    # 1. ベーシックスタイル。 "Series.str" can be used to access the values of the series as strings and apply several methods to it.
    df1_1 = df.str.extract(r'(?P<basic_ymd_1>(?P<bm_1>\d{1,2}[/-])(?P<bd>\d{1,2}[/-])(?P<btdy>\d{2})(?P<bfdy>\d{2})?)')
    df1_2 = df.str.extract(r'(?P<basic_ymd_2>(?P<bm_2>\d{1,2}[/-])(?P<by>\d{4}))') # dd-ddは年月で無いので含めないように分けた

    # 2. 英語表記Month
    df2  = df.str.extract(r'(?P<word_ymd>(?P<wdpre>\d{1,2})?.?(?P<wm>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)(?P<wdpost>\W{0,2}\d{1,2}\w{0,2})?.{1,2}(?P<wy>\d{4}))')

    # 3. 年のみ
    df3 = df.str.extract(r'(?P<year_only>\d{4})')

    # 【2】様々な表記を１つのカラムに集約する
    df_extracted = pd.concat([df1_1, df1_2, df2, df3], axis=1)
    df_extracted['all'] = df.values
    df_extracted['answer_ymd'] = df_extracted.agg(func_agg_various_ymd_str, axis='columns')

    # デバッグ用に調整
#     print(df_extracted.columns)
    df_extracted = df_extracted[['answer_ymd', 'word_ymd', 'year_only']]
        
    # 【3】年月日でソート
    df_extracted['ymd_datetype'] = pd.to_datetime(df_extracted['answer_ymd'], format='%m/%d/%Y')
    df_sorted = df_extracted.sort_values(by='ymd_datetype')
    print(df_sorted)
    print(df_sorted.index)

    return df_sorted.index # Your answer here

df_sorted = date_sorter()


     answer_ymd            word_ymd year_only ymd_datetype
9     4/10/1971                 NaN       NaN   1971-04-10
84    5/18/1971                 NaN       NaN   1971-05-18
2      7/8/1971                 NaN       NaN   1971-07-08
53    7/11/1971                 NaN       NaN   1971-07-11
28    9/12/1971                 NaN       NaN   1971-09-12
474    1/1/1972                 NaN      1972   1972-01-01
153   1/13/1972         13 Jan 1972      1972   1972-01-13
13    1/26/1972                 NaN       NaN   1972-01-26
129   5/06/1972         06 May 1972      1972   1972-05-06
98    5/13/1972                 NaN       NaN   1972-05-13
111   6/10/1972                 NaN       NaN   1972-06-10
225   6/15/1972       June 15, 1972      1972   1972-06-15
31    7/20/1972                 NaN      9079   1972-07-20
171  10/04/1972         04 Oct 1972      1972   1972-10-04
191  11/30/1972         30 Nov 1972      1972   1972-11-30
486    1/1/1973                 NaN      1973   1973-01-

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:38: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:39: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFra